In [5]:
import numpy as np
import matplotlib.pyplot as plt
from keras import utils
from keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras.datasets import cifar10
from keras.datasets import cifar100

In [6]:
dataset1 = cifar10
(x_train1, y_train1), (x_test1, y_test1) = dataset1.load_data() #original size train(xxx, 28, 28),test(xxx)

img = x_test1[0] 
#plt.imshow(img)
#translate the data to image
x_train1 = x_train1.astype("float32")
x_test1 = x_test1.astype("float32")


#normalization
x_train1 /= 255
x_test1 /= 255

vgg_cifar10 = load_model('Model/vgg_cifar10.h5')#load lenet5 model trained with mnist dataset 
vgg_cifar10.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 

In [8]:
beta = 0.04079
N = 600
sample(x_test1[0],y_test1[0],vgg_cifar10,N, beta)

Sampled 734 images, 134 outside beta, 600 inside beta
600 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.04079


0.04079

In [15]:
beta = 0.0406
N = 671
sample(x_test1[0],y_test1[0],vgg_cifar10,N, beta)

671 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.0406


0.0406

In [16]:
beta = 0.0375
N = 783
sample(x_test2[0],y_test2[0],vgg_cifar100,N, beta)

783 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.0375


0.0375

In [7]:
dataset2 = cifar100
(x_train2, y_train2), (x_test2, y_test2) = dataset2.load_data() #original size train(xxx, 28, 28),test(xxx)

img = x_test2[0]        
#translate the data to image
x_train2 = x_train2.astype("float32")
x_test2 = x_test2.astype("float32")


#normalization
x_train2 /= 255
x_test2 /= 255

vgg_cifar100 = load_model('Model/vgg_cifar100.h5')#load lenet5 model trained with mnist dataset 
vgg_cifar100.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 


In [11]:
beta2 = 0.03
N = 600
sample(x_test2[0],y_test2[0],vgg_cifar100,N,beta2)

Beta is reduced by 0.0001, the new beta is 0.0299
Beta is reduced by 0.0001, the new beta is 0.0298
Beta is reduced by 0.0001, the new beta is 0.0297
Beta is reduced by 0.0001, the new beta is 0.0296
Beta is reduced by 0.0001, the new beta is 0.029500000000000002
Beta is reduced by 0.0001, the new beta is 0.029400000000000003
Beta is reduced by 0.0001, the new beta is 0.029300000000000003
Beta is reduced by 0.0001, the new beta is 0.029200000000000004
Beta is reduced by 0.0001, the new beta is 0.029100000000000004
Beta is reduced by 0.0001, the new beta is 0.029000000000000005
Beta is reduced by 0.0001, the new beta is 0.028900000000000006
Beta is reduced by 0.0001, the new beta is 0.028800000000000006
Beta is reduced by 0.0001, the new beta is 0.028700000000000007
Beta is reduced by 0.0001, the new beta is 0.028600000000000007
Beta is reduced by 0.0001, the new beta is 0.028500000000000008
Beta is reduced by 0.0001, the new beta is 0.02840000000000001
Beta is reduced by 0.0001, the ne

0.02820000000000001

In [1]:
def sample(x_test,y_test,model,N,beta):
    
    max_distance = 0  #max safe distance
    count = 0
    count2 = 0
    neighbour_list=[]
    mean = np.zeros(3)  
    cov = np.eye(3)
    label = y_test
    
    while True:
        count2+=1
        p = np.random.multivariate_normal(mean, cov,(32,32)) #sample formula
        neighbour = p + x_test*255
        neighbour /= 255
        neighbour=neighbour.reshape(1,32,32,3)
        distance=np.linalg.norm(p[0]) 
        distance /= 255
        #print(distance)
        if (distance < beta or distance == beta):
            
            output = model.predict(neighbour,batch_size=128)
            proba = softmax(output)
            pred = np.argmax(proba)
            if(pred == label): 
                count+=1  #correct prediction
            else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                beta = beta-0.0001
                print("Beta is reduced by 0.0001, the new beta is",beta)   
            
        if(count==N):  
            #print("Sampled %s images, %s outside beta, %s inside beta" % (count2, count2-count,count))
            print("%s images inside beta have the same label, local robustness formula satisfied" %N)
            break

  
    print("max safe distance:",beta)            

    return beta

In [8]:
def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()

In [12]:
beta = 0.0265
N = 671
sample(x_test2[0],y_test2[0],vgg_cifar100,N, beta)

671 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.0265


0.0265

In [13]:
beta = 0.0225
N = 783
sample(x_test2[0],y_test2[0],vgg_cifar100,N, beta)

783 images inside beta have the same label, local robustness formula satisfied
max safe distance: 0.0225


0.0225